<a href="https://colab.research.google.com/github/eduardoseity/Loterias/blob/main/Notebooks/Loterias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Olá,
## Este notebook foi criado com o objetivo de analisar os resultados das loterias e exercitar alguns conceitos estatísticos.

Técnicas e conceitos aplicados:
- Raspagem de dados
- Orientação a objetos utilizando a biblioteca ABC do python
- Análise exploratória dos dados
- Estatística

# Importando as bibliotecas 📚 necessárias

In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import os
import urllib3
from abc import ABC, abstractmethod
import random
import numpy as np
import plotly.express as px

urllib3.disable_warnings()

# Criando as pastas 📂 necessárias

In [2]:
if not os.path.exists('assets/results'):
  os.makedirs('assets/results')

# Raspando os dados do site https://loterias.caixa.gov.br 🌐

In [3]:
# Criando uma sessão do requests
session = requests.Session()

In [4]:
def get_results(modalidade:str, overwrite:bool=False):
  '''
  Executa a raspagem de dados no site e salva o resultado no diretório "assets/results/"

  :param modalidade: `str` Modalidade da loteria
  :param overwrite: `bool` Indica se o resultado deve ser sobrescrito ou não caso já exista um arquivo gerado na data atual
  '''
  print('Iniciando processo de raspagem...')

  # Salva a data atual
  date = datetime.now()
  # Configura nome do arquivo
  filename = f'assets/results/{modalidade}.csv'
  # Verifica se arquivo já existe na pasta
  if os.path.exists(filename):
      # Pega a data de criação do arquivo existente
      file_date = datetime.fromtimestamp(os.path.getmtime(filename))
      # Retorna caso a data atual coincida com a data de criação do arquivo
      if file_date.date() == date.date() and not overwrite:
        print('Um arquivo com a data de hoje já existe na pasta /assets/results')
        return

  # Realiza a requisição no site
  print('Requisição iniciada')
  re = session.get(f'https://servicebus2.caixa.gov.br/portaldeloterias/api/resultados?modalidade={modalidade}', verify=False)
  # Transforma o resultado em Json
  j = json.loads(re.text)
  # Cria um objeto BeautifulSoup com o Json
  soup = BeautifulSoup(j['html'], 'html.parser')
  # Seleciona todas as tags <table>
  for index, tag in enumerate(soup.select('table')):
      # Exclui tabelas excedentes
      if index > 0: tag.extract()
  # Transforma a tabela em um DataFrame do pandas
  table = pd.read_html(str(soup))[0]
  # Exporta o DataFrame para csv
  table.to_csv(filename, index=False)
  print(f'Processo finalizado. Resultado salvo em {filename}')

# Orientação a objetos 🗃


## Interfaces
### Contém os métodos e atributos básicos para uma Loteria 🍀

In [5]:
class ILotto(ABC):
    @abstractmethod
    def min_bet(self): # Quantidade de números mínimo para aposta <int>
        pass
    @abstractmethod
    def max_bet(self): # Quantidade de números máximo permitido para aposta <int>
        pass
    @abstractmethod
    def prices(self): # Preços das apostas <list>
        pass
    @abstractmethod
    def winning_categories(self): # Faixas de acertos válidos <list>
        pass
    @abstractmethod
    def prizes(self): # Faixas de premiações <list>
        pass
    @abstractmethod
    def lower_upper_number(self): # Maior dezena <int>
        pass
    @abstractmethod
    def modalidade(self): # Modalidade da loteria <str>
        pass
    @abstractmethod
    def drawn_number_columns(self): # Número das colunas que contém as bolas sorteadas <list>
        pass
    @abstractmethod
    def results_dataframe(self): # DataFrame contendo os resultados obtidos <pandas.DataFrame>
        pass
    @abstractmethod
    def drawn_numbers_qty(self): # Quantidade de números sorteados <int>
        pass
    @abstractmethod
    def numbers_position(self): # Matriz contendo a disposição dos números no volante <list>
        pass

    @abstractmethod
    def validate_bet(self, bet:list) -> bool:
        '''
        Retorna `True` se a aposta for válida
        '''
        if len(bet) < self.min_bet or len(bet) > self.max_bet: # Testa se a quantidade de números apostados está dentro dos limites
            raise ValueError(f'Quantidade de números apostados deve estar entre {self.min_bet} e {self.max_bet}')
        elif len(set(bet)) != len(bet): # Verifica se não há números repetidos
            raise ValueError('Existem números repetidos na aposta')
        elif sorted(bet)[0] < self.lower_upper_number[0] or sorted(bet)[-1] > self.lower_upper_number[1]: # Verifica se os números estão dentro do intervalo permitido
            raise ValueError('Existem números fora do intervalo permitido')
        else:
            return True

    def check_result(self, result:np.ndarray, bet_numbers:list) -> list:
        '''
        Compara uma aposta com um resultado e retorna uma lista contendo os números acertados
        '''
        drawn_numbers = result[self.drawn_number_columns] # Números sorteados
        winner_numbers = [x for x in bet_numbers if x in drawn_numbers] # Itera entre os números apostados e retorna o número caso esteja entre os números sorteados
        return winner_numbers

    def check_results(self, results_array:np.ndarray, bet_numbers:list) -> list:
        '''
        Compara uma lista de apostas com um array de resultados e retorna uma lista contendo os números acertados
        '''
        if not isinstance(bet_numbers[0], list): raise TypeError('O parâmetro bet_numbers deve ser uma lista de listas')
        winner_numbers = []
        for r in results_array: # Itera entre os resultados
            for b in bet_numbers: # Itera entre as apostas
                winner_numbers.append(self.check_result(r, b)) # Realiza a comparação
        return winner_numbers

    @abstractmethod
    def update_results(self, overwrite:bool = False) -> pd.DataFrame:
        '''
        Atualiza os resultados
        '''
        get_results(self.modalidade, overwrite)
        self.results_dataframe = pd.read_csv(f'assets/results/{self.modalidade}.csv')
        return self.results_dataframe

    def generate_random(self, num:int) -> list:
        '''
        Gera números aleatórios
        '''
        if num < self.min_bet or num > self.max_bet: raise ValueError(f"O parâmetro 'num' deve ser um número entre {self.min_bet} e {self.max_bet}") # Verifica se a quantidade solicitada é permitida
        remaining_numbers = list(range(self.lower_upper_number[0], self.lower_upper_number[1]+1)) # Cria uma lista com todos os números possíveis
        bet = []
        for x in range(num):
            index = random.randint(0, len(remaining_numbers)-1) # Escolhe um index aleatório
            bet.append(remaining_numbers[index]) # Adiciona o número escolhido
            remaining_numbers.remove(bet[-1]) # Remove o número escolhido da lista de números possíveis

        return bet

    def get_last_result(self) -> np.ndarray:
        '''
        Retorna o último resultado
        '''
        self.update_results()
        return self.results_dataframe[-1:].to_numpy()

    def get_results(self, interval:list=None, update:bool=False) -> np.ndarray:
        '''
        Retorna um intervalo de resultados
        '''
        if update: self.update_results()
        if interval != None:
            return self.results_dataframe.iloc[interval].to_numpy()
        else:
            return self.results_dataframe.to_numpy()

## Classes
### Contém uma classe para cada modalidade de Loteria 🍀

### Megasena

In [6]:
class Megasena(ILotto):
    def __init__(self) -> None:
        self.name = 'Megasena'

    @property
    def min_bet(self):
        return 6
    @property
    def max_bet(self):
        return 20
    @property
    def winning_categories(self):
        return [4,5,6]
    @property
    def prices(self):
        return [5, 35, 140, 420, 1050, 2310, 4620, 8580, 15015, 25025, 40040, 61880, 92820, 135660, 193800]
    @property
    def prizes(self):
        return []
    @property
    def lower_upper_number(self):
        return (1,60)
    @property
    def modalidade(self):
        return 'Mega-Sena'
    @property
    def drawn_number_columns(self):
        return [2,3,4,5,6,7]
    @property
    def results_dataframe(self):
        return self._results_dataframe
    @results_dataframe.setter
    def results_dataframe(self, value:pd.DataFrame()):
        self._results_dataframe = value
    @property
    def drawn_numbers_qty(self):
        return 6
    @property
    def numbers_position(self):
        return [
            list(range(1,11)),
            list(range(11,21)),
            list(range(21,31)),
            list(range(31,41)),
            list(range(41,51)),
            list(range(51,61)),
        ]

    def update_results(self, overwrite: bool = False) -> pd.DataFrame:
        return super().update_results(overwrite)

    def validate_bet(self, bet: list):
        return super().validate_bet(bet)

### Lotofácil

In [7]:
class Lotofacil(ILotto):

    def __init__(self) -> None:
        self.name = 'Lotofacil'

    @property
    def min_bet(self):
        return 15
    @property
    def max_bet(self):
        return 20
    @property
    def winning_categories(self):
        return [11,12,13,14,15]
    @property
    def prices(self):
        return [3, 48, 408, 2448, 11628, 46512]
    @property
    def prizes(self):
        return []
    @property
    def lower_upper_number(self):
        return (1,25)
    @property
    def modalidade(self):
        return 'Lotofácil'
    @property
    def drawn_number_columns(self):
        return [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
    @property
    def results_dataframe(self):
        return self._results_dataframe
    @results_dataframe.setter
    def results_dataframe(self, value:pd.DataFrame()):
        self._results_dataframe = value
    @property
    def drawn_numbers_qty(self):
        return 15
    @property
    def numbers_position(self):
        return [
            list(range(1,6)),
            list(range(6,11)),
            list(range(11,16)),
            list(range(16,21)),
            list(range(21,26)),
        ]

    def update_results(self, overwrite: bool = False) -> pd.DataFrame:
        return super().update_results(overwrite)

    def validate_bet(self, bet: list):
        return super().validate_bet(bet)

### Quina

In [8]:
class Quina(ILotto):

    def __init__(self) -> None:
        self.name = 'Quina'

    @property
    def min_bet(self):
        return 5
    @property
    def max_bet(self):
        return 15
    @property
    def winning_categories(self):
        return [2,3,4,5]
    @property
    def prices(self):
        return [2.5,15,52.5,140,315,630,1155,1980,3217.5,5005,7507.5]
    @property
    def prizes(self):
        return []
    @property
    def lower_upper_number(self):
        return (1,80)
    @property
    def modalidade(self):
        return 'Quina'
    @property
    def drawn_number_columns(self):
        return [2,3,4,5,6]
    @property
    def results_dataframe(self):
        return self._results_dataframe
    @results_dataframe.setter
    def results_dataframe(self, value:pd.DataFrame()):
        self._results_dataframe = value
    @property
    def drawn_numbers_qty(self):
        return 5
    @property
    def numbers_position(self):
        return [
            list(range(1,11)),
            list(range(11,21)),
            list(range(21,31)),
            list(range(31,41)),
            list(range(41,51)),
            list(range(51,61)),
            list(range(61,71)),
            list(range(71,81))
        ]

    def update_results(self, overwrite: bool = False) -> pd.DataFrame:
        return super().update_results(overwrite)

    def validate_bet(self, bet: list):
        return super().validate_bet(bet)

### Lotomania

In [9]:
class Lotomania(ILotto):

    def __init__(self) -> None:
        self.name = 'Lotomania'

    @property
    def min_bet(self):
        return 50
    @property
    def max_bet(self):
        return 50
    @property
    def winning_categories(self):
        return [0,15,16,17,18,19,20]
    @property
    def prices(self):
        return [3]
    @property
    def prizes(self):
        return []
    @property
    def lower_upper_number(self):
        return (1,100)
    @property
    def modalidade(self):
        return 'Lotomania'
    @property
    def drawn_number_columns(self):
        return [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
    @property
    def results_dataframe(self):
        return self._results_dataframe
    @results_dataframe.setter
    def results_dataframe(self, value:pd.DataFrame()):
        self._results_dataframe = value
    @property
    def drawn_numbers_qty(self):
        return 20
    @property
    def numbers_position(self):
        return [
            list(range(1,11)),
            list(range(11,21)),
            list(range(21,31)),
            list(range(31,41)),
            list(range(41,51)),
            list(range(51,61)),
            list(range(61,71)),
            list(range(71,81)),
            list(range(81,91)),
            list(range(91,101)),
        ]

    def update_results(self, overwrite: bool = False) -> pd.DataFrame:
        df = super().update_results(overwrite)
        # Muda zero por 100 (a tabela original utiliza os dois casos, mas no jogo eles são considerados os mesmos números)
        df.iloc[:,self.drawn_number_columns] = df.iloc[:,self.drawn_number_columns].replace([0],[100])
        return df

    def validate_bet(self, bet: list):
        return super().validate_bet(bet)

# Estatísticas 📊

In [10]:
def StdDev(data:np.ndarray):
    '''
    Calcula os desvios padrões dos resultados e exibe um histograma com os resultados
    '''
    std_dev = []
    for d in data:
        std_dev.append(np.std(d))
    ax = px.histogram(x=std_dev, title='Contagem de desvio padrão', text_auto=True, labels={'x':'Desvio padrão'}) \
        .update_traces(marker_line_width=1) \
        .update_layout(yaxis_title='Ocorrências')

    ax.show()

def NumberCount(data:np.ndarray):
    '''
    Realiza a contagem de todas as dezenas sorteadas
    '''
    ax = px.histogram(y=np.ravel(data), title='Contagem de dezenas sorteadas', text_auto=True, labels={'y':'Dezena'}) \
        .update_layout(xaxis_title='Ocorrências') \
        .update_traces(marker_line_width=1)

    ax.show()

def EvenCount(data:np.ndarray):
    '''
    Realiza a contagem da quantidade de números pares sorteados em cada resultado
    '''
    even_count = np.array([])
    for d in data:
        even = [x for x in d if x%2 == 0]
        even_count = np.append(even_count,len(even))

    even_unique = np.unique(even_count).astype(int)
    bins = int(even_unique.max() - even_unique.min() + 1)
    percentage = np.around((np.histogram(even_count, bins=bins)[0]/len(even_count))*100,2)
    ax = px.histogram(even_count, title='Contagem de números pares', text_auto=True, labels={'value':'Quantidade de pares'}, nbins=bins) \
        .update_layout(yaxis_title='Ocorrências') \
        .update_traces(marker_line_width=1, customdata=percentage, hovertemplate='<b>%{customdata}%</b><br><extra></extra>')
    ax.show()

# Playground 🎢

In [11]:
#@title ## Escolha a Loteria
#@markdown Depois de selecionar clique no botão Play
loteria = "Megasena" #@param ["Megasena", "Lotofácil", "Quina", "Lotomania"]
print(f'Loteria {loteria} selecionada!')
print()

loto = None
# Instancia a Loteria selecionada
match loteria:
  case 'Megasena': loto = Megasena()
  case 'Lotofácil': loto = Lotofacil()
  case 'Quina': loto = Quina()
  case 'Lotomania': loto = Lotomania()
  case _:
    print('Loteria inválida!')

# Salva os resultados de todos os sorteios
if loto != None:
  results = loto.get_results(update=True)[:][:,loto.drawn_number_columns]

Loteria Megasena selecionada!

Iniciando processo de raspagem...
Requisição iniciada
Processo finalizado. Resultado salvo em assets/results/Mega-Sena.csv


## Abaixo estão algumas visualizações para entender o conjunto de dados

### Exibe a contagem de quantas vezes cada número foi sorteado

In [12]:
NumberCount(results)

Com base na análise visual é possível dizer que algum número possui vantagem sobre os outros?

### Calcula o desvio padrão do conjunto de números sorteados de cada concurso e plota em um histograma

In [13]:
StdDev(results)

O desvio padrão mede a dispersão do conjunto de dados, ou seja, quanto maior for o valor do desvio padrão mais distante da média os dados estarão.

No gráfico acima a região da esquerda demonstra resultados dos quais os números sorteados não estão muito dispersos, ou seja, é como se os números estivessem aglomerados, próximos uns dos outros.

Já na região da direita os números sorteados estão mais dispersos, ou seja, existe um espaçamento maior entre os números.

Pode-se observar que a maior concentração está na região central onde os números sorteados não estão muito distantes uns dos outros e nem muito aglomerados.

### Conta a quantidade de números pares sorteados em cada concurso e plota em um histrograma

In [14]:
EvenCount(results)

## Teste sua sorte 🤞

In [15]:
#@title ## Digite a sua aposta abaixo

#@markdown Os números devem estar separados por vírgula

Aposta = ',11,51,32,  7,46,58,' #@param {type:"string"}

aposta = Aposta.replace(' ','').split(',')
aposta = [int(x) for x in aposta if x != '']
_ = loto.validate_bet(aposta)

In [16]:
print('Comparando a sua aposta com todos os resultados...')
hist = loto.get_results()
print('Total de sorteios realizados:', hist.shape[0])
print('Números apostados:', sorted(aposta))
matches = loto.check_results(hist, [aposta])
print()
max = 0
for m in matches:
  if len(m) > max: max = len(m)
print('Máximo número de acertos:', max)


Comparando a sua aposta com todos os resultados...
Total de sorteios realizados: 2608
Números apostados: [7, 11, 32, 46, 51, 58]

Máximo número de acertos: 4
